# Preamble

In [1]:
import glob
import os
import pandas as pd
import numpy as np
import seaborn as sns
import wikitextparser as wtp
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
from datetime import datetime
from ast import literal_eval

# Lists

In [2]:
# API variables
api_url = 'https://yugipedia.com/api.php'
sets_query_url = '?action=ask&query=[[Category:Set%20Card%20Lists]]|limit%3D5000|order%3Dasc&format=json'
lists_query_url = '?action=query&prop=revisions&rvprop=content&format=json&titles='

rarity_dict = {'c': 'Common', 'r': 'Rare', 'sr': 'Super Rare', 'ur': 'Ultra Rare', 'utr': 'Ultimate Rare', 'n': 'Normal', 'nr': 'Normal Rare', 'sp': 'Short Print', 'ssp': 'Super Short Print', 'hfr': 'Holofoil Rare', 'scr': 'Secret Rare', 'uscr': 'Ultra Secret Rare', 'scur': 'Secret Ultra Rare', 'pscr': 'Prismatic Secret Rare', 'hgr': 'Holographic Rare', ' gr': 'Ghost Rare', 'pr': 'Parallel Rare', 'npr': 'Normal Parallel Rare', 'pc': 'Parallel Common', 'spr': 'Super Parallel Rare', 'upr': 'Ultra Parallel Rare', 'dnpr': 'Duel Terminal Normal Parallel Rare', 'dpc': 'Duel Terminal Parallel Common', 'drpr': 'Duel Terminal Rare Parallel Rare', 'dspr': 'Duel Terminal Super Parallel Rare', 'dupr': 'Duel Terminal Ultra Parallel Rare', 'DScPR': 'Duel Terminal Secret Parallel Rare', 'gur': 'Gold Rare', 'escr': 'Extra Secret Rare', 'ggr': 'Ghost/Gold Rare', 'shr': 'Shatterfoil Rare', 'cr': 'Collector\'s Rare', 'str': 'Starlight Rare', 'gr': 'Ghost Rare', 'gscr': 'Gold Secret Rare', 'sfr': 'Starfoil Rare'}
regions_dict = {'EN':'English','E':'English', 'PT': 'Portuguese', 'P': 'Portuguese', 'DE': 'German', 'G': 'German', 'FR': 'French', 'F': 'French', 'IT': 'Italian', 'I': 'Italian', 'SP': 'Spanish', 'S': 'Spanish', 'JP': 'Japanese', 'AE': 'Asian English', 'KR': 'Korean', 'K': 'Korean', 'TC': 'Traditional Chinese', 'SC': 'Simplified Chinese'}

In [17]:
test = pd.read_json('https://yugipedia.com/api.php?action=askargs&conditions=Legend%20of%20Blue%20Eyes%20White%20Dragon&printouts=English%20release%20date&format=json')

In [16]:
test2 = pd.DataFrame(test['query']['results']).T
test3 = pd.DataFrame(test2['printouts'].values.tolist())
timestamp = test3['English release date'][0][0]['timestamp']
pd.Timestamp(int(timestamp), unit='s').ctime()

'Fri Mar  8 00:00:00 2002'

In [6]:
df = pd.read_json(f'{api_url}{sets_query_url}')
keys = list(df['query']['results'].keys())
print(len(keys))

3963


In [7]:
all_set_lists = pd.DataFrame(columns = ['Set','Card number','Name','Rarity','Print','Quantity'])

for i in range(40):
    first = i*100
    last = (i+1)*100
    titles = '|'.join(keys[first:last]).replace(' ','%20').encode('utf-8')
    df = pd.read_json(f'{api_url}{lists_query_url}{titles}')
    contents = df['query']['pages'].values()
    
    for content in contents:
        if 'revisions' in  content.keys():
            temp = content['revisions'][0]['*']
            parsed = wtp.parse(temp)
            
            for template in parsed.templates:
                if template.name == 'Set list':
                    title = content['title'].split('Lists:')[1]
                    set_df = pd.DataFrame(columns = ['Set', 'Card number', 'Name', 'Rarity', 'Print', 'Quantity'])


                    region = None
                    rarity = None
                    card_print = None
                    qty = None
                    desc = None
                    opt = None
                    list_df = None
                    
                    for argument in template.arguments:
                        if 'region=' in argument:
                            region = argument.string[argument.string.index('=')+1:]
                        elif 'rarities=' in argument:
                            rarity = tuple(rarity_dict.get(i.strip().lower(), i.strip()) for i in argument.string[argument.string.index('=')+1:].split(','))
                        elif 'print=' in argument:
                            card_print = argument.string[argument.string.index('=')+1:]
                        elif 'qty=' in argument:
                            qty = argument.string[argument.string.index('=')+1:]
                        elif 'description=' in argument:
                            desc = argument.string[argument.string.index('=')+1:]
                        elif 'options=' in argument:
                            opt = argument.string[argument.string.index('=')+1:]
                        else:
                            set_list = argument.string[2:-1]
                            lines = set_list.split('\n')

                            list_df = pd.DataFrame([x.split(';') for x in lines])
                            list_df = list_df[~list_df[0].str.contains('!:')]
                            list_df = list_df.applymap(lambda x: x.split('//')[0] if x is not None else x)
                            list_df = list_df.applymap(lambda x: x.strip() if x is not None else x)
                            list_df.replace(r'^\s*$', None, regex = True, inplace = True)

                    if opt != 'noabbr':
                        set_df['Card number'] = list_df[0]
                        set_df['Name'] = list_df[1]
                    else: 
                        set_df['Name'] = list_df[0]

                    if len(list_df.columns)>2: # and rare in str
                        set_df['Rarity'] = list_df[2].apply(lambda x: tuple([rarity_dict.get(y.strip().lower(), y.strip()) for y in x.split(',')]) if x is not None else rarity)
                    else:
                        set_df['Rarity'] = [rarity for _ in set_df.index]

                    if len(list_df.columns)>3 :
                        if card_print is not None: # and new/reprint in str
                            set_df['Print'] = list_df[3].apply(lambda x: x if x is not None else card_print)
                            if len(list_df.columns)>4 and qty is not None:
                                set_df['Quantity'] = list_df[4].apply(lambda x: x if x is not None else qty)
                        elif qty is not None:
                            set_df['Quantity'] = list_df[3].apply(lambda x: x if x is not None else qty)
                    
                    set_df['Set'] = title
                    all_set_lists = pd.concat([all_set_lists, set_df], ignore_index=True)
                    
        else:
            print('Error:', content['title'])

Error: B"Set Card Lists:Duelist Pack Collection Tin 2008 (TCG-EN)
Error: Set Card Lists:Sneak Peek Participation Cards: Series 5 (TCG-IT)
Error: Set Card Lists:Expert Edition Volume.4 (OCG-KR)
Error: B"Set Card Lists:Structure Deck: Warrior's Triumph (TCG-FR)
Error: B"Set Card Lists:Yu-Gi-Oh! World Championship Tournament 2004 promotional cards (TCG-NA)
Error: Set Card Lists:All Fusion Monsters (WC6-EN)
Error: B"Set Card Lists:Duelist Pack: Battle City (TCG-PT)
Error: Set Card Lists:Sneak Peek Participation Cards: Series 5 (TCG-SP)
Error: Set Card Lists:2-Player Starter Deck: Yuya
Error: B"Set Card Lists:Duelist League Demo 2010 (TCG-SP)
Error: Set Card Lists:Yu-Gi-Oh: Super Fusion! Bonds That Transcend Time Movie Pack (OCG-KR)
Error: Set Card Lists:Sneak Peek Participation Cards: Series 2 (TCG-EN)
Error: B"Set Card Lists:Yu-Gi-Oh! Championship Series 2014 pre-registration card (TCG-EN)
Error: Set Card Lists:Sneak Peek Participation Cards: Series 3 (TCG-SP)
Error: B"Set Card Lists:Duel

In [8]:
all_set_lists

,Set,Card number,Name,Rarity,Print,Quantity
0,2015 Mega-Tins (TCG-FR),CT12-FR001,Odd-Eyes Pendulum Dragon,"(Platinum Secret Rare,)",NaN,NaN
1,2015 Mega-Tins (TCG-FR),CT12-FR003,Elder Entity Norden,"(Super Rare,)",NaN,NaN
2,2015 Mega-Tins (TCG-FR),CT12-FR004,Majesty's Fiend,"(Super Rare,)",NaN,NaN
3,2015 Mega-Tins (TCG-FR),CT12-FR002,Dark Rebellion Xyz Dragon,"(Platinum Secret Rare,)",NaN,NaN
4,2015 Mega-Tins (TCG-FR),CT12-FR005,Old Entity Hastorr,"(Super Rare,)",NaN,NaN
...,...,...,...,...,...,...
72279,Invasion of Chaos: Special Edition (TCG-NA),IOC-SE2,Lava Golem,"(Ultra Rare,)",NaN,NaN
72280,Invasion of Chaos: Special Edition (TCG-NA),IOC-SE3,Magic Cylinder,"(Ultra Rare,)",NaN,NaN
72281,Invasion of Chaos: Special Edition (TCG-NA),IOC-SE4,Ring of Destruction,"(Ultra Rare,)",NaN,NaN
72282,Invasion of Chaos promotional cards (OCG-KR),IOC-KRPR1,Mystical Refpanel,"(Secret Rare,)",NaN,NaN


In [9]:
all_set_lists.groupby('Card number', dropna=False).nunique()

,Set,Name,Rarity,Print,Quantity
Card number,,,,,
19YJ-JPT01,1,1,1,0,0
2010-AE003,1,1,1,0,0
2010-EN001,1,1,1,0,0
2010-EN002,1,1,1,0,0
2012-AE003,1,1,1,0,0
...,...,...,...,...,...
ZTIN-FR021,1,1,1,0,0
ZTIN-FRV01,1,1,1,0,0
ZTIN-FRV02,1,1,1,0,0


In [10]:
all_set_lists.groupby('Name', dropna=False).nunique()

,Set,Card number,Rarity,Print,Quantity
Name,,,,,
"""A"" Cell Breeding Device",4,3,1,0,0
"""A"" Cell Incubator",4,4,1,1,0
"""A"" Cell Recombination Device",5,5,1,0,0
"""A"" Cell Scatter Burst",4,4,1,0,0
"""Backup Soldier""",1,1,0,0,0
...,...,...,...,...,...
Zubaba Knight,24,24,5,1,1
Zubababancho Gagagacoat,2,2,1,1,0
"Zure, Knight of Dark World",8,7,1,0,1


In [11]:
all_set_lists.explode('Rarity').groupby('Rarity').nunique()

,Set,Card number,Name,Print,Quantity
Rarity,,,,,
10000 Secret Rare,7,7,1,1,0
20ScR,1,16,16,0,0
20th Secret Rare,12,84,83,2,0
20th Secret rare,1,2,2,0,0
Collector's Rare,6,90,15,2,0
Collectors Rare,3,65,65,1,0
Common,844,35602,6946,3,3
Duel Terminal Normal Parallel Rare,16,526,333,0,0
Duel Terminal Normal Rare Parallel Rare,2,2,2,0,0


In [12]:
all_set_lists.groupby('Print', dropna=False).nunique()

,Set,Card number,Name,Rarity,Quantity
Print,,,,,
,1,166,0,0,0
European debut,5,8,8,2,0
New,161,5643,2057,35,3
New artwork,8,23,8,4,0
North American debut,1,2,2,1,0
Reprint,156,6322,2800,33,3
Reprint (New Art),1,1,1,1,0
Reprint (functional errata),10,12,7,6,0
NaN,1615,59197,10116,104,3


In [13]:
all_set_lists.groupby('Quantity', dropna=False).nunique()

,Set,Card number,Name,Rarity,Print
Quantity,,,,,
1,122,4525,1044,11,2
2,117,437,129,4,2
3,21,31,10,1,2
NaN,1654,66383,11007,112,8


In [14]:
all_set_lists[all_set_lists['Card number']=='!: header::Kaiba Deck']

,Set,Card number,Name,Rarity,Print,Quantity
